In [201]:
import pandas as pd

In [202]:
data1 = pd.read_csv('data/pivotdata.csv', parse_dates=True)
data1[:1]

,Date,Type,Currency,Pnl
0,20170101,Equity,USD,100


In [203]:
data2 = pd.read_csv('data/data.csv', parse_dates=True)
data2[:1]

,Date,Class,Id,Value
0,20170101,A,1,10.0


In [208]:
# pivot data by multiple columns
pivot1 = pd.pivot_table(data1, index=['Date'], columns=['Type','Currency'], values='Pnl', fill_value=0.0).reset_index()
pivot1[:1]

Type          Date Equity      Property    
Currency              GBP  USD      GBP USD
0         20170101     50  100       50  50

In [209]:
# collapse multi level column to single level
pivot1_collapsed = pivot1.copy()
pivot1_collapsed.columns = [pivot1.columns.map('{0[0]}.{0[1]}'.format)]
pivot1_collapsed['Date'] = pivot1_collapsed['Date.']
pivot1_collapsed = pivot1_collapsed.drop(['Date.'],axis=1)
pivot1_collapsed

,Equity.GBP,Equity.USD,Property.GBP,Property.USD,Date
0,50,100,50,50,20170101
1,30,90,30,15,20170102
2,13,20,42,10,20170103


In [210]:
# merge with 2nd table by column
merged = pd.merge(data2,pivot1_collapsed,on='Date')
merged[:1]

,Date,Class,Id,Value,Equity.GBP,Equity.USD,Property.GBP,Property.USD
0,20170101,A,1,10.0,50,100,50,50


In [211]:
# merge with with index
pivot1_collapsed_indexed = pivot1_collapsed.copy()
data2_indexed = data2.copy()
pivot1_collapsed_indexed = pivot1_collapsed_indexed.set_index('Date')
data2_indexed = data2_indexed.set_index('Date')
print(pivot1_collapsed_indexed[:1])
print(data2_indexed[:1])

          Equity.GBP  Equity.USD  Property.GBP  Property.USD
Date                                                        
20170101          50         100            50            50
         Class  Id  Value
Date                     
20170101     A   1   10.0


In [212]:
merged_indexed = pd.merge(pivot1_collapsed_indexed,data2_indexed,left_index=True,right_index=True)
merged_indexed[:2]

,Equity.GBP,Equity.USD,Property.GBP,Property.USD,Class,Id,Value
Date,,,,,,,
20170101,50,100,50,50,A,1,10.0
20170101,50,100,50,50,B,2,50.0


In [213]:
# merge 2 frames with different column level
pivot1_indexed = pivot1.copy()
pivot2_indexed = pd.pivot_table(data, index=['Date'], columns=['Class'], values='Value', fill_value=0.0).reset_index()
pivot1_indexed = pivot1_indexed.set_index('Date')
pivot2_indexed = pivot2_indexed.set_index('Date')
pivot1_indexed[:1]

Type     Equity      Property    
Currency    GBP  USD      GBP USD
Date                             
20170101     50  100       50  50

In [214]:
pivot2_indexed[:1]

Class,A,B,C
Date,,,
20170101,10,50,100


In [215]:
# add 2nd column level to match pivot1
pivot2_indexed.columns = pd.MultiIndex.from_product([['Top'], pivot2_indexed.columns])
pivot2_indexed[:1]

Top         
           A   B    C
Date                 
20170101  10  50  100

In [216]:
merge2_indexed = pd.merge(pivot1_indexed, pivot2_indexed, left_index=True, right_index=True)
merge2_indexed[:2]

Type     Equity      Property     Top         
Currency    GBP  USD      GBP USD   A   B    C
Date                                          
20170101     50  100       50  50  10  50  100
20170102     30   90       30  15  11  49  101

In [217]:
# helpers
print(pivot.columns)
print(pivot.columns.get_level_values(0))
print(pivot.columns.get_level_values(1))
print(pivot['Date'])
print(pivot['Equity']['GBP'])

MultiIndex(levels=[['Equity', 'Property', 'Date'], ['GBP', 'USD', '']],
           labels=[[2, 0, 0, 1, 1], [2, 0, 1, 0, 1]],
           names=['Type', 'Currency'])
Index(['Date', 'Equity', 'Equity', 'Property', 'Property'], dtype='object', name='Type')
Index(['', 'GBP', 'USD', 'GBP', 'USD'], dtype='object', name='Currency')
0    20170101
1    20170102
2    20170103
Name: Date, dtype: int64
0    50
1    30
2    13
Name: GBP, dtype: int64
